In [1]:
import pandas as pd
import os
#mingw_path = 'C:\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

#os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
#import xgboost as xg
import datetime
import sklearn.preprocessing as pp

In [238]:
train = pd.read_csv('CrossSellTarget\\Train.csv')

C:\Continuum\Anaconda2\envs\TFENV\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [239]:
test = pd.read_csv('CrossSellTarget\\Test.csv')

C:\Continuum\Anaconda2\envs\TFENV\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [148]:
train['OCCUP_ALL_NEW'].unique()

array(['SALARIED'], dtype=object)

In [240]:
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns 

In [241]:
mis_val_table_ren_columns = missing_values_table(train)

In [242]:
nan_col = list(mis_val_table_ren_columns[mis_val_table_ren_columns['% of Total Values']> 90].index)

In [243]:
for col in train.columns.values:
    if(len(train[col].unique()) == 1):
        nan_col.append(col)

In [245]:
len(nan_col)

213

In [244]:
train.drop(nan_col,axis=1 , inplace=True)
test.drop(nan_col,axis=1 , inplace=True)

In [171]:
train.to_csv('train_pruned.csv')
test.to_csv('test_pruned.csv')

In [187]:
shutil.make_archive("CrossSellTarget\\PrunedDS", "zip", "CrossSellTarget\\PrunedDS")

'C:\\Continuum\\Anaconda2\\envs\\TFENV\\CrossSellTarget\\PrunedDS.zip'

In [246]:
y_train = train['RESPONDERS']
train.drop(['RESPONDERS'],axis=1,inplace=True)

In [6]:
train.dropna(axis=1,how='all' ,inplace=True)

In [7]:
test.dropna(axis=1,how='all',inplace=True)

In [247]:
len(train.columns)

126

In [248]:
len(test.columns)

126

In [249]:
for i in train.columns:
    if(i not in test.columns):
        print (i)

In [181]:
train.drop(['ZIP_CODE_FINAL'],axis=1 , inplace=True)
test.drop(['ZIP_CODE_FINAL'],axis=1,inplace= True)

In [250]:
for i in train.columns:
    if len(train[i].astype('category').cat.categories.get_values()) <= 3:
        train[i] = train[i].astype('category').cat.codes

In [251]:
for i in test.columns:
    if len(test[i].astype('category').cat.categories.get_values()) <= 3:
        test[i] = test[i].astype('category').cat.codes

In [17]:
train['MATURITY_GL_YR'] = train['MATURITY_GL'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else -1 )
train['MATURITY_GL_MN'] = train['MATURITY_GL'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else -1 )


train['MATURITY_LAP_YR'] = train['MATURITY_LAP'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else -1 )
train['MATURITY_LAP_MN'] = train['MATURITY_LAP'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else -1 )


train['MATURITY_LAS_YR'] = train['MATURITY_LAS'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else -1 )
train['MATURITY_LAS_MN'] = train['MATURITY_LAS'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else -1 )


train['CLOSED_DATE_YR'] = train['CLOSED_DATE'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else -1 )
train['CLOSED_DATE_MN'] = train['CLOSED_DATE'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else -1 )

In [18]:
test['MATURITY_GL_YR'] = test['MATURITY_GL'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else -1 )
test['MATURITY_GL_MN'] = test['MATURITY_GL'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else -1 )


test['MATURITY_LAP_YR'] = test['MATURITY_LAP'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else -1 )
test['MATURITY_LAP_MN'] = test['MATURITY_LAP'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else -1 )


test['MATURITY_LAS_YR'] = test['MATURITY_LAS'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else -1 )
test['MATURITY_LAS_MN'] = test['MATURITY_LAS'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else -1 )


test['CLOSED_DATE_YR'] = test['CLOSED_DATE'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else -1 )
test['CLOSED_DATE_MN'] = test['CLOSED_DATE'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else -1 )

In [ ]:
train.drop(['MATURITY_LAS','CLOSED_DATE','MATURITY_LAP','MATURITY_GL'],axis=1,inplace=True)
test.drop(['MATURITY_LAS','CLOSED_DATE','MATURITY_LAP','MATURITY_GL'],axis=1,inplace=True)

In [254]:
cat_cols=['LEGAL_ENTITY','DESIGNATION_FINAL',
          'PA_PQ_TAG','NEFT_CC_CATEGORY','NEFT_DC_CATEGORY','TPT_DC_CATEGORY_MON_01',
'TPT_CC_CATEGORY_MON_01','IMPS_CC_CATEGORY_MON_01'
    #,'CLOSED_DATE_YR','CLOSED_DATE_MN','MATURITY_LAP_YR','MATURITY_LAP_MN',
#'MATURITY_LAS_YR','MATURITY_LAS_MN','MATURITY_GL_YR','MATURITY_GL_MN'
]

In [253]:
for i in train.columns:
    if (len(train[i].astype('category').cat.categories.get_values()) > 3) & (str(train[i].dtype) == 'object'):
        print (i)

LEGAL_ENTITY
DESIGNATION_FINAL
PA_PQ_TAG
NEFT_CC_CATEGORY
NEFT_DC_CATEGORY
TPT_DC_CATEGORY_MON_01
TPT_CC_CATEGORY_MON_01
IMPS_CC_CATEGORY_MON_01
ZIP_CODE_FINAL


In [255]:
for col in cat_cols:
    print (col)
    train[col] = train[col].apply(lambda x: x if str(x)  != str(pd.np.nan) else '-1' )
    test[col] = test[col].apply(lambda x: x if str(x)  != str(pd.np.nan) else '-1' )
    train[col] = train[col].astype('str')
    test[col] = test[col].astype('str')
    data= test[col].append(train[col])
    data = data.astype('str')
    le = pp.LabelEncoder().fit(data)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
    

LEGAL_ENTITY
DESIGNATION_FINAL
PA_PQ_TAG
NEFT_CC_CATEGORY
NEFT_DC_CATEGORY
TPT_DC_CATEGORY_MON_01
TPT_CC_CATEGORY_MON_01
IMPS_CC_CATEGORY_MON_01


In [261]:
zip_tr= train['ZIP_CODE_FINAL']
zip_ts= test['ZIP_CODE_FINAL']
train.drop(['ZIP_CODE_FINAL'] , inplace = True , axis = 1)
test.drop(['ZIP_CODE_FINAL'] , inplace = True , axis = 1)

In [262]:
le = pp.LabelEncoder().fit(y_train)
y_train = le.transform(y_train)
y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [263]:
for col in train.columns.values:
    train[col] = train[col].apply(lambda x: x if str(x) != str(pd.np.nan) else -1)
    test[col] = test[col].apply(lambda x: x if str(x) != str(pd.np.nan) else -1)
    print (col)

CUSTOMER_ID
ACT_TYPE
GENDER
LEGAL_ENTITY
AGE
N_CASA_MAX_BALANCE_MTD
N_CASA_MIN_BALANCE_MTD
CC_HOLD_MON_03
CC_HOLD_MON_02
CC_HOLD_MON_01
CC_ACTIVE_MON_03
CC_ACTIVE_MON_02
CC_ACTIVE_MON_01
CC_HOLD
CC_ACTIVE
DC_HOLD
DC_ACTIVE_MON_03
DC_ACTIVE_MON_02
DC_ACTIVE_MON_01
DC_ACTIVE
DC_SPEND_MON_03
DC_SPEND_MON_02
DC_SPEND_MON_01
DC_SPEND_MON_04
CR_LIMIT
STMT_MON_03
STMT_MON_02
STMT_MON_01
COC_ELIGIBLE
EOP_BAL_MON_01
AMB_MON_04
AMB_MON_01
AMB_MON_02
AMB_MON_03
NB_MON_01_CNT
NB_MON_02_CNT
NB_MON_03_CNT
CC_TXN_MON_01
CC_SPEND_MON_06
CC_TXN_MON_06
CC_TXN_MON_03
CC_TXN_MON_02
CC_TXN_MON_04
CC_SPEND_MON_05
CC_TXN_MON_05
DC_TXN_MON_04
DC_TXN_MON_01
DC_SPEND_MON_06
DC_TXN_MON_06
DC_SPEND_MON_05
DC_TXN_MON_05
DC_TXN_MON_03
DC_TXN_MON_02
DC_SPEND_MON_04_GRC
DC_SPEND_MON_04_RST
FINANCE_MON_01
JOBS_MON_01
LIFESTYLE_MON_01
INMON_01KET_MON_01
TRAVEL_MON_01
CHANNEL_CLICK_DISP
DESIGNATION_FINAL
TOP14_CITY
TOP9_CITY
PL2CC_ELIGIBLE
D_AMT_L3_MON_06
C_AMT_L3_MON_06
C_COUNT_L3_MON_06
D_COUNT_L3_MON_06
MAX_C_AMT_L3_

In [188]:
#from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

In [264]:
train.index = train['CUSTOMER_ID']
train.drop(['CUSTOMER_ID'],axis=1,inplace=True)

In [265]:
test.index = test['CUSTOMER_ID']
test.drop(['CUSTOMER_ID'],axis=1,inplace=True)

In [266]:
cor = train.corr()

In [272]:
cols=[]
i=0
for col in cor.columns.values:
    for index in cor[col].index:
        if(cor.get_value(index,col) > 0.80 or cor.get_value(index,col) < -0.80) and (index != col ) and sorted((index,col)) not in cols:
            i+=1
            tpl = (index,col)
            cols.append(sorted(tpl))
            print ( i,sorted(tpl),'=> ',cor.get_value(index,col))
            

1 ['D_AMT_L3_MON_06', 'N_CASA_MAX_BALANCE_MTD'] =>  0.829014096056
2 ['C_AMT_L3_MON_06', 'N_CASA_MAX_BALANCE_MTD'] =>  0.830185265317
3 ['MAX_C_AMT_L3_MON_06', 'N_CASA_MAX_BALANCE_MTD'] =>  0.833647124703
4 ['D_AMT_L3_MON_01', 'N_CASA_MAX_BALANCE_MTD'] =>  0.814700543364
5 ['C_AMT_L3_MON_01', 'N_CASA_MAX_BALANCE_MTD'] =>  0.834152782045
6 ['MAX_C_AMT_L3_MON_01', 'N_CASA_MAX_BALANCE_MTD'] =>  0.900534805716
7 ['NEFT_CC_AMOUNT', 'N_CASA_MAX_BALANCE_MTD'] =>  0.833201381124
8 ['CC_HOLD_MON_02', 'CC_HOLD_MON_03'] =>  0.966432097425
9 ['CC_HOLD_MON_01', 'CC_HOLD_MON_03'] =>  0.926765221005
10 ['CC_HOLD', 'CC_HOLD_MON_03'] =>  0.961387826261
11 ['CC_HOLD_MON_01', 'CC_HOLD_MON_02'] =>  0.959511418914
12 ['CC_HOLD', 'CC_HOLD_MON_02'] =>  0.96426963006
13 ['CC_ACTIVE', 'CC_HOLD_MON_02'] =>  0.805015766607
14 ['CC_HOLD', 'CC_HOLD_MON_01'] =>  0.967632601229
15 ['CC_ACTIVE', 'CC_HOLD_MON_01'] =>  0.821596284287
16 ['CC_ACTIVE_MON_02', 'CC_ACTIVE_MON_03'] =>  0.830637104
17 ['CC_ACTIVE_MON_01', 'C

In [273]:
cols_0= [i[0] for i in cols]
cols_1= [i[1] for i in cols]

In [274]:
#cols_0 = list(set(cols_0))
cols_0.sort()
#cols_1 = list(set(cols_1))
cols_1.sort()

In [275]:
col_rem=[]
for col in cols_0:
    if col in cols_1:
        col_rem.append(col)
col_rem = list(set(col_rem))

cols_0 = list(set(cols_0))
for col in col_rem:
    if col in cols_0:
        cols_0.remove(col)
len(col_rem)

24

In [276]:
train.drop(col_rem, axis=1,inplace=True)
test.drop(col_rem,axis=1,inplace=True)

In [277]:
len(cols_0)

18

In [278]:
train.drop(cols_0, axis=1,inplace=True)
test.drop(cols_0,axis=1,inplace=True)

In [280]:
train.shape

(300000, 82)

In [281]:
test.shape

(200000, 82)

In [282]:
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

In [283]:
x_train = train.as_matrix()
x_test = test.as_matrix()

In [284]:
pca = PCA(n_components=3)

In [285]:
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [286]:
score = pca.explained_variance_ratio_
V = pca.components_

In [287]:
x_train_trans = pca.transform(x_train)
x_test_trans = pca.transform(x_test)

In [288]:
model = LogisticRegression()
#model = SGDClassifier()
#model = LogisticRegressionCV()
#model = RandomForestClassifier(max_features = None , n_jobs=2)

In [289]:
model.fit(x_train_trans,y_train)
#model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [290]:
y_pred=model.predict(x_test_trans)
#y_pred=model.predict(x_test)

In [132]:
#y_pred = le.inverse_transform(y_pred)

In [291]:
y_pred = pd.DataFrame(y_pred,columns=['RESPONDERS'])
y_pred.index = test.index

In [292]:
import shutil
y_pred.to_csv('CrossSellTarget\\submission\\sample_submission.csv')
shutil.make_archive("CrossSellTarget\\submission", "zip", "CrossSellTarget\\submission")

'C:\\Continuum\\Anaconda2\\envs\\TFENV\\CrossSellTarget\\submission.zip'